<a href="https://colab.research.google.com/github/shanglapk/NFRDeepLearningFramework/blob/main/ReqiRoberta1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# We won't need TensorFlow here
#!pip uninstall -y tensorflow
# Install `transformers` from master
#!pip install git+https://github.com/huggingface/transformers
#!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.11.0
# tokenizers version at notebook update --- 0.8.0rc1
!pip install transformers
from tokenizers import ByteLevelBPETokenizer
import os
import time
# Initialize a tokenizer
# Record the start time
start_time = time.time()
tokenizer = ByteLevelBPETokenizer()
paths = ["/content/Req2VecCorpusOSM.txt"]  # Replace with the actual file path on your computer
tokenizer.train(files=paths, vocab_size=52000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
    "",
])
# Calculate the elapsed time
elapsed_time = time.time() - start_time

print(f"Training completed in {elapsed_time:.2f} seconds.")
tokenizer.save_model( "/content/drive/MyDrive/SoftwareRequirement/tokenizer_output","en")

In [ ]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
tokenizer = ByteLevelBPETokenizer(
    "/content/drive/MyDrive/SoftwareRequirement/tokenizer_output/en-vocab.json",
    "/content/drive/MyDrive/SoftwareRequirement/tokenizer_output/en-merges.txt",
)
tokenizer._tokenizer.post_processor = BertProcessing(
    ("", tokenizer.token_to_id("")),
    ("", tokenizer.token_to_id("")),
)
tokenizer.enable_truncation(max_length=514)
print(tokenizer.encode("The message must be updated for all users in a chat within 0.1 seconds, given that all users are online and have LTE connection or better."))
tokenizer.encode("The message must be updated for all users in a chat within 0.1 seconds, given that all users are online and have LTE connection or better.").tokens

In [ ]:
!pip install transformers[torch] accelerate==0.20.1
import torch
from transformers import RobertaConfig, RobertaForMaskedLM, RobertaTokenizer
from transformers import LineByLineTextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

# Define your model configuration
config = RobertaConfig(
    vocab_size=50265,
    hidden_size=768,
    type_vocab_size=1,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=12,
    intermediate_size=4096,
    hidden_dropout_prob=0.1,
    attention_probs_dropout_prob=0.1,
    layer_norm_eps=1e-5,
    use_cache=True,
    num_labels=2,
    output_attentions=False,
    output_hidden_states=False,

)

# Initialize your model
model = RobertaForMaskedLM(config=config)

# Initialize the tokenizer
tokenizer = RobertaTokenizer(
    vocab_file="/content/drive/MyDrive/SoftwareRequirement/tokenizer_output/en-vocab.json",# Output of the trained tokenizer
    merges_file = "/content/drive/MyDrive/SoftwareRequirement/tokenizer_output/en-merges.txt",#Output of the trained tokenizer
    max_len=514,
)

# Define your training data
train_file = "/content/Req2VecCorpusOSM.txt"

# Create a dataset from your text data
dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path=train_file,
    block_size=514,
)
# Define the path to your evaluation text file
eval_file = "/content/Req2VecCorpus.txt"

# Create an evaluation dataset from your text data
eval_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path=eval_file,
    block_size=514,
)

# Define a data collator for language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,  # Set mlm to True for masked language modeling
)

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/SoftwareRequirement/training_output",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    evaluation_strategy="steps",
    eval_steps=5_000,
    learning_rate=2e-5,
    warmup_steps=500,  # Number of warm-up steps for the learning rate scheduler
    weight_decay=0.01,  # L2 regularization term
    logging_dir="/content/drive/MyDrive/SoftwareRequirement/training_logs",  # Logging directory
    logging_steps=100,  # How often training logs are written
    gradient_accumulation_steps=1,  # Accumulate gradients over multiple steps
    seed=42,  # Random seed for reproducibility
    label_smoothing_factor=0.1,  # Label smoothing factor
    report_to="tensorboard",  # Report results to TensorBoard
)

# Initialize the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    eval_dataset=eval_dataset,
)

# Start training
trainer.train()

# Save the trained model
model.save_pretrained("/content/drive/MyDrive/SoftwareRequirement/models/version1/ReqiRoberta")
